## Cleaning Data

In [9]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.cdc.gov", None)

results = client.get("8xkx-amqh", date = "2023-04-19T00:00:00.000", limit = 99999999)

# Convert to pandas DataFrame
raw0 = pd.DataFrame.from_records(results)
for row in range(raw0.shape[0]): 
    seperated=raw0.loc[row,'recip_county'].split(' ')
    toadd = raw0.loc[row,'recip_state'] + " "
    for countyname in seperated:
        if countyname != seperated[-1]: toadd += countyname
        try:
            if countyname != seperated[-2]: toadd += " "
        except:
            pass
    raw0.loc[row,'recip_county'] = toadd
raw0 = raw0.drop(['date', 'fips', 'mmwr_week','recip_state'], axis=1)
print(raw0.head())

   recip_county completeness_pct administered_dose1_recip  \
0  IN Randolph              98.5                    13253   
1      MO Polk              91.3                    14811   
2    TN Hardin              97.7                    12252   
3    TN Cannon              97.7                     5576   
4      NE Holt                91                      NaN   

  administered_dose1_pop_pct administered_dose1_recip_5plus  \
0                       53.7                          13234   
1                       46.1                          14795   
2                       47.8                          12242   
3                         38                           5571   
4                        NaN                            NaN   

  administered_dose1_recip_5pluspop_pct administered_dose1_recip_12plus  \
0                                    57                           13002   
1                                    49                           14438   
2                            

In [16]:
import numpy as np 
import pandas as pd 
import math 
from time import sleep
import time
import os
import requests
from bs4 import BeautifulSoup
import lxml
import html5lib
from pandas import json_normalize
%matplotlib inline

raw1 = pd.read_csv('https://data.cdc.gov/resource/k8wy-p9cg.csv?$query=SELECT%0A%20%20%60data_as_of%60%2C%0A%20%20%60start_week%60%2C%0A%20%20%60end_week%60%2C%0A%20%20%60state%60%2C%0A%20%20%60county_name%60%2C%0A%20%20%60urbanruralcode%60%2C%0A%20%20%60fipsstate%60%2C%0A%20%20%60fipscounty%60%2C%0A%20%20%60fipscode%60%2C%0A%20%20%60indicator%60%2C%0A%20%20%60all_deaths_total%60%2C%0A%20%20%60covid_19_deaths_total%60%2C%0A%20%20%60non_hispanic_white%60%2C%0A%20%20%60non_hispanic_black%60%2C%0A%20%20%60non_hispanic_american_indian%60%2C%0A%20%20%60non_hispanic_asian%60%2C%0A%20%20%60non_hispanic_nhopi%60%2C%0A%20%20%60hispanic%60%2C%0A%20%20%60other%60%2C%0A%20%20%60urbanruraldesc%60%2C%0A%20%20%60footnote%60%0AWHERE%20%60start_week%60%20%3D%20%222020-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp')
for row in range(raw1.shape[0]): 
    seperated=raw1.loc[row,'county_name'].split(' ')
    if len(seperated) == 2: raw1.loc[row,'county_name'] = raw1.loc[row,'state'] + " " + seperated[0]
    else: 
        toadd = raw1.loc[row,'state'] + " "
        for countyname in seperated:
            if countyname != seperated[-1]: toadd += countyname
            try:
                if countyname != seperated[-2]: toadd += " "
            except:
                pass
        raw1.loc[row,'county_name'] = toadd
    
raw1 = raw1.drop(['state','data_as_of', 'start_week', 'end_week','urbanruralcode','fipsstate','fipscounty','fipscode','indicator','urbanruraldesc','footnote'], axis=1)
print(raw1.shape[0])
print(raw1.head())

1000
                county_name  all_deaths_total  covid_19_deaths_total  \
0              AK Anchorage              8401                    767   
1              AK Anchorage              8401                    767   
2              AK Anchorage              8401                    767   
3  AK Fairbanks North Star               2011                    201   
4  AK Fairbanks North Star               2011                    201   

   non_hispanic_white  non_hispanic_black  non_hispanic_american_indian  \
0               0.570               0.045                         0.214   
1               0.452               0.035                         0.254   
2               0.564               0.052                         0.083   
3               0.701               0.025                         0.182   
4               0.612                 NaN                         0.264   

   non_hispanic_asian  non_hispanic_nhopi  hispanic  other  
0               0.058               0.031     0.03

In [3]:
key="7a1d4f2ad8bdd748802e4dda66a65fe3c9bef3df"

url='https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=county:*&in=state:*&key='+key
response = requests.get(url).json()
raw2 = pd.DataFrame.from_dict(response)
raw2.columns=raw2.iloc[0]
raw2 = raw2.reset_index()
raw2 = raw2.drop(raw2.index[0])

StateAbbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI","Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD","Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO","Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC","South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT","Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC","American Samoa": "AS","Guam": "GU","Northern Mariana Islands": "MP","Puerto Rico": "PR","United States Minor Outlying Islands": "UM","U.S. Virgin Islands": "VI",}
raw2 = raw2.reset_index()
for row in range(raw2.shape[0]):
    seperated=raw2.loc[row,"NAME"].split(' ')
    if not seperated[-1] in StateAbbrev.keys(): toadd = raw2.loc[row,"NAME"]
    else:
        toadd = StateAbbrev[seperated[len(seperated)-1]] + " " + seperated[0]
        for i in seperated[1:-1]:
            if i != "County" and i != "County,": toadd += " " + i
    
    raw2.loc[row,'NAME'] = toadd
    
raw2 = raw2.drop(['index','level_0','state','county'], axis=1)

print(raw2.head())

0         NAME    POP
0   MS Coahoma  22124
1    MS Jasper  16383
2     MS Jones  68098
3  MS Walthall  14286
4    MS Monroe  35252


In [4]:
raw3 = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/counties/asrh/cc-est2021-agesex-all.csv', encoding='latin-1')
raw3 = raw3.loc[raw3['YEAR'] == 2]
raw3 = raw3.reset_index()
for row in range(raw3.shape[0]):
    seperated=raw3.loc[row,'CTYNAME'].split(' ')
    try:
        raw3.loc[row,'CTYNAME'] = StateAbbrev[raw3.loc[row,'STNAME']] + " " + seperated[0]
    except:
        raw3.drop([row])
raw3 = raw3.drop(['SUMLEV','STATE','COUNTY','STNAME','index','YEAR'], axis=1)
print(raw3.head())

      CTYNAME  POPESTIMATE  POPEST_MALE  POPEST_FEM  UNDER5_TOT  UNDER5_MALE  \
0  AL Autauga        58877        28734       30143        3480         1812   
1  AL Baldwin       233140       113612      119528       12176         6348   
2  AL Barbour        25180        13373       11807        1342          694   
3     AL Bibb        22223        11901       10322        1219          594   
4   AL Blount        59081        29400       29681        3474         1808   

   UNDER5_FEM  AGE513_TOT  AGE513_MALE  AGE513_FEM  ...  AGE7579_FEM  \
0        1668        7042         3606        3436  ...         1058   
1        5828       25601        13101       12500  ...         4892   
2         648        2675         1322        1353  ...          532   
3         625        2210         1159        1051  ...          376   
4        1666        6910         3480        3430  ...         1168   

   AGE8084_TOT  AGE8084_MALE  AGE8084_FEM  AGE85PLUS_TOT  AGE85PLUS_MALE  \
0         

In [5]:
import xlrd
import openpyxl

raw4 = pd.read_excel('https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.xlsx?v=6499.5', skiprows=[0,1,2,3])
#raw4 = raw4.drop([0,1,2,3])
droplist=[]
raw4 = raw4.reset_index()
for row in range(raw4.shape[0]):
    seperated=raw4.loc[row,'Area_name'].split(' ')
    if "County" in seperated:
        try:
            raw4.loc[row,'Area_name'] = raw4.loc[row,'Stabr'] + " " + seperated[0]
        except:
            droplist.append(row)
    else: 
        droplist.append(row)

raw4 = raw4.drop(droplist, axis=0)
raw4 = raw4.drop(['index','FIPS_code','Stabr'], axis=1)
print(raw4.head())

    Area_name  Rural-urban_Continuum_Code_2003  Urban_Influence_Code_2003  \
2  AL Autauga                              2.0                        2.0   
3  AL Baldwin                              4.0                        5.0   
4  AL Barbour                              6.0                        6.0   
5     AL Bibb                              1.0                        1.0   
6   AL Blount                              1.0                        1.0   

   Rural-urban_Continuum_Code_2013  Urban_Influence_Code_2013  POVALL_2020  \
2                              2.0                        2.0       6242.0   
3                              3.0                        2.0      20189.0   
4                              6.0                        6.0       5548.0   
5                              1.0                        1.0       3549.0   
6                              1.0                        1.0       7525.0   

   CI90LBALL_2020  CI90UBALL_2020  PCTPOVALL_2020  CI90LBALLP_2020  

In [6]:
import numpy as np 
import pandas as pd 
import math 
from time import sleep
import time
import os
import requests
from bs4 import BeautifulSoup
import lxml
import html5lib
%matplotlib inline

columns=[]
columns.append(['raw1',['covid_19_deaths_total']])

iteration=0
colcount=0
allcolumns=[]
for columnlists in [i for i in [raw0.columns[1:],raw2.columns[1:],raw3.columns[1:],raw4.columns[1:]]]: 
    
    if iteration == 1: iteration = 2
    columnstemp=[]
    for column in columnlists:
        columnstemp.append(str(column))
        colcount+=1
    
    columns.append([str("raw"+str(iteration)), columnstemp])
    for columntemp in columnstemp: allcolumns.append(str(columntemp))
    
    iteration+=1
    
#print(columns)
df = pd.DataFrame(columns = allcolumns)

In [8]:
print([0,raw0.shape[0],1,raw1.shape[0],2,raw2.shape[0],3,raw3.shape[0],4,raw4.shape[0]])

[0, 1000, 1, 1000, 2, 3220, 3, 3143, 4, 3007]


In [7]:
def whichdf(column):
            
    for dfandcolumn in columns:

        if column in dfandcolumn[1]: return dfandcolumn[0]
    
raw0 = raw0.reset_index()
raw1 = raw1.reset_index()
raw2 = raw2.reset_index()
raw3 = raw3.reset_index()
raw4 = raw4.reset_index()

def addrow():
    
    global df
    global row
    
    rowtoadd=[]
    
    #replace with correct df (the one that's the longest)
    location = str(raw2.loc[row,'NAME'])
    
    #You can remove the longest df's search
    raw0row = "empty"
    for row2 in range(raw0.shape[0]):
        location2 = str(raw0.loc[row2,'recip_county'])
        if location2 == location: raw0row = row2
    raw1row = "empty"
    for row2 in range(raw1.shape[0]):
        location2 = str(raw1.loc[row2,'county_name'])
        if location2 == location: raw1row = row2
    raw3row = "empty"
    for row2 in range(raw3.shape[0]):
        location2 = str(raw3.loc[row2,'CTYNAME'])
        if location2 == location: raw3row = row2
    raw4row = "empty"
    for row2 in range(raw4.shape[0]):
        location2 = str(raw4.loc[row2,'Area_name'])
        if location2 == location: raw4row = row2
    
    for column in list(df.columns):
        
        column=str(column)
        
        correctdf = whichdf(column)
        
        if correctdf == "raw0" and not raw0row == "empty": rowtoadd.append(raw0.loc[raw0row, column])
        elif correctdf == "raw1" and not raw1row == "empty": rowtoadd.append(raw1.loc[raw1row, column])
        elif correctdf == "raw2": rowtoadd.append(raw2.loc[row, column])
        elif correctdf == "raw3" and not raw3row == "empty": rowtoadd.append(raw3.loc[raw3row, column])
        elif correctdf == "raw4" and not raw4row == "empty": rowtoadd.append(raw4.loc[raw4row, column])
        else: rowtoadd.append("none")
        
    df.loc[len(df)] = rowtoadd
    
    if row % 1000 == 0 and row != 0: print("finished", row, "of", raw2.shape[0])

#replace with longest df
for row in range(raw2.shape[0]-1): addrow()

print(df.head())
#df.to_pickle("df.pkl")

finished 1000 of 3220
finished 2000 of 3220
finished 3000 of 3220
  completeness_pct administered_dose1_recip administered_dose1_pop_pct  \
0             none                     none                       none   
1             none                     none                       none   
2             none                     none                       none   
3             none                     none                       none   
4             none                     none                       none   

  administered_dose1_recip_5plus administered_dose1_recip_5pluspop_pct  \
0                           none                                  none   
1                           none                                  none   
2                           none                                  none   
3                           none                                  none   
4                           none                                  none   

  administered_dose1_recip_12plus administer

## Initial OLS

In [ ]:
from statsmodels.compat import lzip
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import statsmodels.formula.api as smf

formulas = ['deathspercap ~ dose1', 'deathspercap ~ series', 'deathspercap ~ booster', 'deathspercap ~ dose1 + percapincome', 'deathspercap ~ series + percapincome', 'deathspercap ~ booster + percapincome']

def olsmaker(formula):
    
    OLS = smf.ols(formula, data=df).fit()
    
    print("FORMULA:", formula, "\n\nAIC is", OLS.aic, "\nBIC is", OLS.bic, "\nR-Squared is", OLS.rsquared_adj, "\n"); print(OLS.summary()); print("\n")
    
    fig = sm.graphics.plot_fit(OLS,1, vlines=False); plt.show()
    fig = sm.graphics.plot_partregress_grid(OLS); plt.show()
    fig = sm.graphics.plot_ccpr_grid(OLS); plt.show()

In [ ]:
olsmaker(formulas[0])

In [ ]:
olsmaker(formulas[1])

In [ ]:
olsmaker(formulas[2])

In [ ]:
olsmaker(formulas[3])

In [ ]:
olsmaker(formulas[4])

In [ ]:
olsmaker(formulas[5])

## LASSO

In [ ]:
dfcopy = df.iloc[:,1:]
Y = dfcopy.loc[:, "deathspercap"]
Xlist = []

droplist = [["dose1",["deaths","deathspercap","county","index", "deaths", "series", "booster","pop"]],["series",["deathspercap","deaths","county","index", "deaths", "dose1", "booster","pop"]],["booster",["deathspercap","deaths","county","index", "deaths", "dose1", "series","pop"]]]

for typeanddrops in droplist:
    
    xtype = typeanddrops[0]
    dfcopy = df
    dfcopy = dfcopy.drop(typeanddrops[1], axis=1)
    
    Xlist.append([xtype, dfcopy, list(dfcopy.columns)])

In [ ]:
from sklearn.linear_model import lasso_path
from itertools import cycle
from sklearn.preprocessing import StandardScaler #usually, we want to standardize to put in terms of standard deviations. to minimize iimpact of measurement
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoLarsCV, LassoLarsIC
import time

SC = StandardScaler()

Xns = []
for X in Xlist:
    SC.fit(X[1]) 
    Xns.append(SC.transform(X[1])) #use these statistic to standardize. Xn contains standardized x variables

def lasso(Xnum):
    print("Primary X Value:", Xlist[Xnum][0])
    # Use "eps" to specify the length and density of the grid (eps = alpha_min / alpha_max)
    eps=2.220446049250313e-16
    alphas_lasso, coefs_lasso, _ = lasso_path(Xns[Xnum], Y, eps = eps, max_iter=1000000000) #first, we need to select lamda values
    #eps is one of the parameters to use for finding which lamda values to use
    #eps chooses smallest lamda value
    #eps = lamda min/ lamda max
    #can use n_alphas to select number of alphas in regularization path
    #alphas_lasso is lamdas that were used, coef is lasso estimates
    # Display the solution path
    plt.figure(figsize=(8, 7), dpi=80) #define figure size
    colors = cycle(['b', 'r', 'g', 'c','m', 'y', 'k']) #coloring lines. iterates colors. k is black

    # Take the log of the alpha values to adjust the scale of X-axis
    log_alphas_lasso = np.log10(alphas_lasso) #taking log of lamdas
    
    varname = Xlist[Xnum][2]
    # Use a for-loop to plot several paths on a figure 
    for coef_l, c, vn in zip(coefs_lasso, colors, varname): #zip holds these functions temperarly. coeff lasso is parameter estimate, colors is the colors, varname is variable name.
        l1 = plt.plot(log_alphas_lasso, coef_l, c=c, label=vn) #drawing lines. coefff lasso starts with first row. 
        #in first iteragtion, selects first row of this object. then first item in color. variable name will be variable name.

    plt.xlabel('Log(alpha)')
    plt.ylabel('Coefficient Estimtes')
    plt.title('Lasso Solution Path')
    plt.legend()
    plt.show()
    #log alpha negative because they're small numbers.
    
def lassopick(Xnum, cvoraicbic):
    
    print("Primary X Value:", Xlist[Xnum][0])
    varname = Xlist[Xnum][2]
    
    eps=2.220446049250313e-16
    
    if cvoraicbic == "cv":
        print("PICKING BEST MODEL BASED ON CROSS VALIDATION\n")
        t1 = time.time() # Get the current time
        lascv = LassoLarsCV(cv=5).fit(Xns[Xnum], Y) #how many chunks you break the data into. use 5 or 10 folds, include x and y
        t_lasso_lars_cv = time.time() - t1 # Calculate running time

        # Display the results
        lascv_log_alphas = np.log10(lascv.cv_alphas_ + eps)  #result variable.cv alphas is lamdas. alpha (singular) is lamda that is optimal. eps is different from what we used before. eps is just small number. include zero because taki9ng log of zero creates error
        lascv_log_alpha = np.log10(lascv.alpha_) #log of lamda that minimized MSE
        # Caution: lascv.alphas contains the alpha at the lowest MSE whereas lascv.cv_alphas_ contains the set of alphas used in the path
        # The smallest value in lascv.cv_alphas_ is 0 so we add eps (a small number) to avoid log(0)

        plt.figure(figsize=(8, 7), dpi=80)
        plt.plot(lascv_log_alphas, lascv.mse_path_.mean(axis=1), 'k',
                 label='Average of the MSEs over the Folds', linewidth=2)
        plt.axvline(lascv_log_alpha, linestyle='--', color='k', #add verticle line at optimal lamda
                    label='alpha selected by CV')

        plt.legend()

        plt.xlabel('Log(alpha)')
        plt.ylabel('Mean Square Error')
        plt.title('Model Selection by Cross Validation (train time: %.2fs)'
                  % t_lasso_lars_cv)
        plt.show()

        print("Best Alpha (Based on Cross Validation):", lascv_log_alpha)
        print("\nBest Model (Based on Cross Validation):")
        for varnum in range(len(varname)): print(varname[varnum],lascv.coef_[varnum])
    
    if cvoraicbic == "aicbic":
        print("PICKING BEST MODEL BASED ON AIC AND BIC\n")
        
        t1 = time.time() # Get the current time
        lascv = LassoLarsCV(cv=5).fit(Xns[Xnum], Y) #how many chunks you break the data into. use 5 or 10 folds, include x and y
        t_lasso_lars_cv = time.time() - t1 # Calculate running time

        # Display the results
        lascv_log_alphas = np.log10(lascv.cv_alphas_ + eps)  #result variable.cv alphas is lamdas. alpha (singular) is lamda that is optimal. eps is different from what we used before. eps is just small number. include zero because taki9ng log of zero creates error
        lascv_log_alpha = np.log10(lascv.alpha_) #log of lamda that minimized MSE
        # Caution: lascv.alphas contains the alpha at the lowest MSE whereas lascv.cv_alphas_ contains the set of alphas used in the path
        # The smallest value in lascv.cv_alphas_ is 0 so we add eps (a small number) to avoid log(0)


        lasic_bic = LassoLarsIC(criterion='bic').fit(Xns[Xnum], Y) #specify which criterian. plug in x and y. store results in these two variables
        lasic_aic = LassoLarsIC(criterion='aic').fit(Xns[Xnum], Y)

        # Display results
        # make a fn to produce figures with the same features repeatedly
        def plot_ic_criterion(model, name, color): 
            alpha_ = model.alpha_ + eps #optimal lamda
            alphas_ = model.alphas_ + eps #lamdas that were used in this function
            criterion_ = model.criterion_ # BIC or AIC values over the alpha values
            plt.plot(np.log10(alphas_), criterion_, '--', color=color,
                     linewidth=3, label='%s' % name)
            plt.axvline(np.log10(alpha_), color=color, linewidth=3, #adding vertical line
                        label='$\lambda$ selected by %s ' % name)
            plt.xlabel('Log($\lambda$)')
            plt.ylabel('Criterion Value')


        plt.figure(figsize=(8, 7), dpi=80)
        plot_ic_criterion(lasic_aic, 'AIC', 'b') #line is AIC result
        plot_ic_criterion(lasic_bic, 'BIC', 'r') #line is BIC result
        plt.legend()
        plt.title('Model Selection by Information Criteria')
        plt.show()

        print("\n\nBest Model (Based on AIC):")
        for varnum in range(len(varname)): print(varname[varnum],lasic_aic.coef_[varnum])
        print("\n\nBest Model (Based on BIC):")
        for varnum in range(len(varname)): print(varname[varnum],lasic_bic.coef_[varnum])

In [ ]:
lasso(0)

In [ ]:
lassopick(0, "cv")

In [ ]:
lassopick(0, "aicbic")

In [ ]:
lasso(1)

In [ ]:
lassopick(1, "cv")

In [ ]:
lassopick(1, "aicbic")

In [ ]:
lasso(2)

In [ ]:
lassopick(2, "cv")

In [ ]:
lassopick(2, "aicbic")

## Forwards Selection

In [ ]:
def forwards(Xlistnum, criteria):
    
    global selected
             
    Xvalues=Xlist[Xlistnum]
    mode = {"AIC":2,"BIC":1,"R-Squared":0}[criteria]
    
    print("TYPE:", Xvalues[0], "CRITERIA:", criteria, "\n")
    
    X = Xvalues[1]
    Xnp = Xvalues[1].values
    ncol=X.shape[1]
    pcand = list(range(ncol)) # A list to keep track of the remaining predictors (i.e., not yet added) at each iteration
    psel = [] # A list to keep track of the selected predictors at each iteration (the order of the selected predictors)
    tb = np.zeros(ncol) # A vector to store the BIC of the selected model (combination) at each iteration
    p = 0 # Iteration idex
    addedlist=[]

    while len(psel) != ncol: # Repeat below until the model includes all the predictors
        
        tb0 = np.zeros((len(pcand),3)) # Store the Rsquare(s) and BIC(s) of the models considered at each iteration

        for i in range(0,len(pcand)):
            psel0 = psel + [pcand[i]] # "psel0" is a temporary version of psel which includes one of the predictors in pcan and those in psel
            # Caution: "+" combines two lists, but not a list and an integer (i.e pcan[i])
            XX = Xnp[:,psel0]
            XX = sm.add_constant(XX)
            model = sm.OLS(Y, XX)
            res = model.fit()
            tb0[i,:] = [res.rsquared, res.bic, res.aic]

        ind = np.argmax(tb0[:,mode]) if mode == 0 else np.argmin(tb0[:,mode])
        addedlist.append(pcand[ind])
        psel = psel + [pcand[ind]] # Add the selected regressor to psel
        pcand.remove(pcand[ind]) # Remove the selected regressor from pcand
        tb[p] =  tb0[ind,1] # Store the BIC of the selected model at this iteration
        p += 1
        
    #[print("\nMODEL NUMBER:", i, "\nThis Model Added Variable:", X.columns[addedlist[i]], "\nCriteria", criteria.upper(), "Value:", tb[i]) for i in range(len(tb))]
        
    best = np.argmax(tb)+1 if mode == 0 else np.argmin(tb)+1
    
    selected = [X.columns[i] for i in list(addedlist[:best])]
    print("SELECTED VARIABLES:", [i for i in selected])

    formula = 'deathspercap ~ '
    for variable in selected:
        formula += " +" + variable

    OLS = smf.ols(formula, data=df).fit()
    print("AIC is", OLS.aic)
    print("BIS is", OLS.bic)
    print("R-Squared is", OLS.rsquared_adj)
    print(OLS.summary())

In [ ]:
forwards(0,"AIC")

In [ ]:
forwards(0,"BIC")

In [ ]:
forwards(0,"R-Squared")

In [ ]:
forwards(1,"AIC")

In [ ]:
forwards(1,"BIC")

In [ ]:
forwards(1,"R-Squared")

In [ ]:
forwards(2,"AIC")

In [ ]:
forwards(2,"BIC")

In [ ]:
forwards(2,"R-Squared")

## Backward Selection

In [ ]:
def backwards(Xnum, criteria):
    
    Xvalues = Xlist[Xnum]
    print("TYPE:", Xvalues[0], "\n")
        
    X = Xvalues[1]
    Xnp = Xvalues[1].values
    ncol=X.shape[1]

    pcand = list(range(ncol)) # A list to keep track of the remaining predictors (i.e., not yet removed) at each iteration
    psel = list(range(ncol)) # A list to keep track of the selected predictors at each iteration (the order of the selected predictors)
    tb = np.zeros(ncol) # A vector to store the AIC, BIC, or R-Squared of the selected model (combination) at each iteration
    p = 0 # Iteration idex
    removelist, fittedlist = [], []

    mode = {"AIC":2,"BIC":1,"R-Squared":0}[criteria]

    print("SELECTING MODEL BASED ON:", criteria)

    while p != ncol: # Repeat below until the model excludes all the predictors

        tb0 = np.zeros((len(pcand),3)) # Store the Rsquared(s), AIC(s),  and BIC(s) of the models considered at each iteration
        fittedlisttemp=[]

        for i in range(0,len(pcand)):

            #Try removing a variable and store resulting AIC, BIC, or R-Squared 
            pseltemp=psel
            pseltemp.remove(pcand[i])
            res = sm.OLS(Y, sm.add_constant(X.iloc[:, pseltemp])).fit()
            tb0[i,:] = [res.rsquared, res.bic, res.aic]
            fittedlisttemp.append(res)
            psel.insert(0, pcand[i])
            psel.sort()

            #print("\nTried removing the following variable:", pcand[i], "\nincluded predictors:", pseltemp, "\nAIC:", tb0[i,2], "BIC:", tb0[i,1], "R-Squared:", tb0[i,0])

        # Find the regressor that results in the best criteria when added to the model
        ind = np.argmax(tb0[:,mode]) if mode == 0 else np.argmin(tb0[:,mode])
        psel.remove(pcand[ind])
        p += 1

        #print("\nRemoving", pcand[ind], "\nWe have gone through the while statement", p, "time(s)\n--------------------------------")

        removelist.append(pcand[ind])
        pcand.remove(pcand[ind]) # Remove the selected regressor from pcand
        tb[p-1] =  tb0[ind,mode] # Store the AIC, BIC, or R-Squared of the selected model at this iteration
        fittedlist.append(fittedlisttemp[ind])

    #Finding Best Model
    best = np.argmax(tb)+1 if mode == 0 else np.argmin(tb)+1

    #[print("\nMODEL NUMBER:", i, "\nThis Model Removed Variable:", removelist[i], "\nCriteria", criteria.upper(), "Value:", tb[i]) for i in range(len(tb))]

    selected = [X.columns[i] for i in list(removelist[best:])]

    formula = 'deathspercap ~ '
    for variable in selected:
        formula += " +" + variable

    OLS = smf.ols(formula, data=df).fit()
    print("AIC is", OLS.aic)
    print("BIS is", OLS.bic)
    print("R-Squared is", OLS.rsquared_adj)
    print(OLS.summary())

In [ ]:
backwards(0, "AIC")

In [ ]:
backwards(0, "BIC")

In [ ]:
backwards(0, "R-Squared")

In [ ]:
backwards(1, "AIC")

In [ ]:
backwards(1, "BIC")

In [ ]:
backwards(1, "R-Squared")

In [ ]:
backwards(2, "AIC")

In [ ]:
backwards(2, "BIC")

In [ ]:
backwards(2, "R-Squared")

## K-Means Clustering

In [ ]:
Xclustering=df[['dose1', 'series','booster']]
Xclustering

In [ ]:
from sklearn.cluster import KMeans
KMres = KMeans(n_clusters=3, random_state=0).fit(Xclustering)
y_pred = KMres.predict(Xclustering)

# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
plt.scatter(Xclustering.iloc[:,0], Xclustering.iloc[:,1], c=y_pred) 

    
plt.xlabel('Dose 1 Percentage ',fontsize = 15)
plt.ylabel('Series Percentage',fontsize = 15)
plt.title('Groups by Dose 1 Percentage and Series Percentage',fontsize = 15)
    
plt.show()

In [ ]:
plt.figure(figsize=(18, 18))

for i in range(1,5):
    plt.subplot(2,2,i)
    KMres = KMeans(n_clusters=i+1, random_state=0).fit(Xclustering)
    y_pred = KMres.predict(Xclustering)

    plt.scatter(Xclustering.iloc[:,0], Xclustering.iloc[:,1], c=y_pred) 
    plt.xlabel('Dose 1 Percentage ',fontsize = 15)
    plt.ylabel('Series Percentage',fontsize = 15)

plt.show()

### Dendogram

In [ ]:
from scipy.cluster import hierarchy

Z = hierarchy.linkage(Xclustering, 'complete') 

plt.figure(figsize=(25, 10))
hierarchy.dendrogram(Z,)
                     
plt.title('Hierarchical Clustering on Vaccine Administered Type', size=20)           
plt.show()

In [ ]:
plt.figure(figsize=(25, 10))
hierarchy.dendrogram(Z,
                     truncate_mode='level', # The other option: 'lastp' - show only p branches
                     p = 5, # number of clusters
                     leaf_rotation=90,
                     leaf_font_size=12,
                     
                     )
                     
plt.title('Hierarchical Clustering on Vaccine Administered Type with five clusters', size=20)          
plt.show()

In [ ]:
from matplotlib import gridspec

plt.figure(figsize=(10, 10))

i=1

for hiertype in ['single', 'complete', 'average', 'weighted', 'centroid', 'ward']:
    
    plt.subplot(2,3,i)
    
    Z = hierarchy.linkage(Xclustering, hiertype) 

    hierarchy.dendrogram(Z, leaf_rotation=90, leaf_font_size=.2, )

    plt.title(hiertype, size=7)
    
    i+=1

plt.show()

### Running regression based off dummy groups

In [ ]:
import matplotlib.patches as mpatches

def clusterreg(num):
    
    kmeans = KMeans(n_clusters=num, random_state=0).fit(Xclustering)
    labels = list(kmeans.labels_)

    dftemp = df
    dftemp['label'] = labels
    groups = dftemp.groupby('label')

    fig, ax = plt.subplots()
    #ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
    for name, group in groups:
        ax.plot(group.dose1, group.series, marker='o', linestyle='', ms=2, label=name)
    ax.legend()
    plt.xlabel('Dose 1 Percentage ',fontsize = 15)
    plt.ylabel('Series Percentage',fontsize = 15)
    plt.title('Groups by Dose 1 Percentage and Series Percentage',fontsize = 15)
    plt.show()
    
    OLS3 = smf.ols("deathspercap ~ label", data=dftemp).fit()
    print("\n\n\n\nAIC is", OLS3.aic, "\nBIS is", OLS3.bic, "\nR-Squared is", OLS3.rsquared_adj, "\n")
    print(OLS3.summary())

In [ ]:
clusterreg(3)

In [ ]:
clusterreg(5)